# Assignment 1



Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:

*   The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
*   Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
*   More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
*   If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
*   Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
*   In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [ ]:
import pandas as pd # library for data analsysis
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from shapely.geometry import Point
from bs4 import BeautifulSoup
import xml
!pip install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
!pip install folium pandas
import folium
from sklearn.cluster import KMeans
print("Libraries installed")

Libraries installed


*Scrape the List of postal codes of Canada*


In [ ]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resource = requests.get(List_url).text

In [ ]:
soup = BeautifulSoup(resource, 'xml')

In [ ]:
table=soup.find('table')

In [ ]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood
column_names = ['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [ ]:
# Search all the postcode, borough, neighbourhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [ ]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


*Time to clen data and remove rows were **Borough** is 'Not assigned'*

In [ ]:
df=df[df['Borough']!='Not assigned']

In [ ]:
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [ ]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [ ]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [ ]:
df_merge.drop_duplicates(inplace=True)

In [ ]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [ ]:
df_merge.head()

,Postalcode,Borough,Neighbourhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df_merge.shape

(103, 3)